# API Testing Notebook

### Imports

In [9]:
import requests
import datetime
import pytz

import os.path

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2 import service_account

from git import Repo
import yaml
import re
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import json

### Functions

In [10]:

def section_names(headers):
    sections_url = f"https://api.todoist.com/rest/v2/sections?project_id={tokens['todoist_projid']}"
    sc = requests.get(sections_url, headers=headers)
    s = sc.json()
    section_dict = {}
    for section in s:
        section_dict[section["id"]] = section["name"]

    return section_dict

### Secrets

In [11]:
with open('secrets/secrets.json') as secrets_file:
    tokens = json.load(secrets_file)

version = 'testing'

### Todoist

In [29]:
message_list = []
headers = {
    'Authorization': f'Bearer {tokens["todoist1"]}',
}
section_dict = section_names(headers)

query_url = f"https://api.todoist.com/rest/v2/tasks?project_id={tokens['todoist_projid']}"
res = requests.get(query_url, headers=headers)
r = res.json()

for result in r:
    query_url = f"https://api.todoist.com/sync/v9/items/get?item_id={result['id']}"
    res = requests.get(query_url, headers=headers)
    item = res.json()["item"]
    description = item["description"]
    section_id = item["section_id"]
    mtime = result["created_at"]
    parent = result["parent_id"]
    print(result)
    dtime = datetime.datetime.strptime(mtime, "%Y-%m-%dT%H:%M:%S.%fZ")

    if result["priority"] != 1:
        prio_text = '<span style="color: #ba0202;">Priority</span>'
    else:
        prio_text = 'Open'

    if len(description) != 0:
        desc_text = f'<br><span class="desc">{description}</span>'
    else:
        desc_text = ''

    if all([section_id != 0, section_id is not None]):
        sect_id_text = f' under <b>{section_dict[section_id]}</b>'
    else:
        sect_id_text = ''

    m = f"{prio_text} Todoist task{sect_id_text}||{result['content']}{desc_text}"
    message_list.append(
            {"time": dtime, "message": m, "via": "Todoist", "id": f'todoist_{result["id"]}', "mtime": mtime,
             "version": version, "priority": result["priority"]})

{'id': '6897450733', 'assigner_id': None, 'assignee_id': None, 'project_id': '2273569911', 'section_id': '123494547', 'parent_id': None, 'order': 1, 'content': 'Simpler implementation of STRATUM region data, minimum PoC', 'description': '', 'is_completed': False, 'labels': [], 'priority': 1, 'comment_count': 0, 'creator_id': '34598745', 'created_at': '2023-05-21T19:31:14.390266Z', 'due': None, 'url': 'https://todoist.com/showTask?id=6897450733'}
{'id': '6917520931', 'assigner_id': None, 'assignee_id': None, 'project_id': '2273569911', 'section_id': '108515198', 'parent_id': None, 'order': 1, 'content': 'Migrate old VIGOR NN resources (and hopefully find old VHNN code)', 'description': '', 'is_completed': False, 'labels': [], 'priority': 1, 'comment_count': 0, 'creator_id': '34598745', 'created_at': '2023-05-28T21:30:10.684459Z', 'due': None, 'url': 'https://todoist.com/showTask?id=6917520931'}
{'id': '6909683084', 'assigner_id': None, 'assignee_id': None, 'project_id': '2273569911', 's

In [28]:
event_dict = {}
message_list = []

headers = {
    'Authorization': f'Bearer {tokens["todoist1"]}',
}
section_dict = section_names(headers)
query_url = f"https://api.todoist.com/sync/v9/completed/get_all?project_id={tokens['todoist_projid']}"
res = requests.get(query_url, headers=headers)
r = res.json()["items"]
for result in r:
    print(result)
    if f"todoist_{result['id']}" not in event_dict.keys():
        query_url = f"https://api.todoist.com/sync/v9/items/get?item_id={result['task_id']}"
        res = requests.get(query_url, headers=headers)
        try:
            item = res.json()["item"]
        except KeyError:
            description = ''
            section_id = None
        else:
            description = item["description"]
            section_id = item["section_id"]

        if len(description) != 0:
            desc_text = f'<br><span class="desc">{description}</span>'
        else:
            desc_text = ''

        if section_id is not None:
            sect_id_text = f' under <b>{section_dict[section_id]}</b>'
        else:
            sect_id_text = ''

        m = f"Closed Todoist task{sect_id_text}||{result['content']}{desc_text}"
    else:
        m = event_dict[f"todoist_{result['id']}"][1]

    # Thanks to Jon Betts on https://stackoverflow.com/questions/23394608/python-regex-fails-to-identify-markdown-links
    name_regex = "[^]]+"
    url_regex = "http[s]?://[^)]+"
    link_regex = '\[({0})]\(\s*({1})\s*\)'.format(name_regex, url_regex)

    # Thanks to NPE & kubanczyk on https://stackoverflow.com/questions/7191209/re-sub-replace-with-matched-content
    m = re.sub(link_regex, r'<a href="\2">\1</a>', m)

    mtime = result["completed_at"]
    dtime = datetime.datetime.strptime(mtime, "%Y-%m-%dT%H:%M:%S.%fZ")
    message_list.append(
        {"time": dtime, "message": m, "via": "Todoist", "id": f'todoist_{result["id"]}', "mtime": mtime,
         "version": version})

{'completed_at': '2023-05-29T17:30:09.000000Z', 'content': 'no-track: subitem testing', 'id': '6354429467', 'meta_data': None, 'project_id': '2273569911', 'section_id': '109935368', 'task_id': '6919851568', 'user_id': '34598745'}
{'completed_at': '2023-05-29T17:16:13.000000Z', 'content': 'Check for in-progress vs complete ID consistency', 'id': '6354398414', 'meta_data': None, 'project_id': '2273569911', 'section_id': '109935368', 'task_id': '6919803871', 'user_id': '34598745'}
{'completed_at': '2023-05-29T17:13:10.000000Z', 'content': 'no-track: Features', 'id': '6354391418', 'meta_data': None, 'project_id': '2273569911', 'section_id': '109935368', 'task_id': '6919739110', 'user_id': '34598745'}
{'completed_at': '2023-05-28T21:36:56.000000Z', 'content': 'Upload STRATUM .shp and WorldClim GeoTIFF 30s precipitation data to Shared Drive', 'id': '6352338909', 'meta_data': None, 'project_id': '2273569911', 'section_id': '123494547', 'task_id': '6917530031', 'user_id': '34598745'}
{'complet